In [1]:
import pandas as pd
pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.
pd.set_option('display.max_rows', None) ## 모든 열을 출력한다.

In [20]:
from module.db.db_model import DbModel
import pandas as pd
import numpy as np
from pprint import pprint
from module.code_matching.mintit import match_mintit_index
from module.code_matching.joongabi import match_joongabi_index
from module.code_matching.bunjang import match_bunjang_index
from module.code_matching.price_list import match_price_list
from module.code_matching.merge import merge_index

db_model = DbModel("main")

phone_list = db_model.get_table_dataframe("g5_phone_list")

mintit_index = match_mintit_index(db_model,phone_list)

joongabi_index = match_joongabi_index(db_model,phone_list)

bunjang_index = match_bunjang_index(db_model,phone_list)


price_list = match_price_list(db_model,phone_list)

result= merge_index(bunjang_index,joongabi_index,mintit_index,price_list,phone_list)

# db_model.update_code_matching(result)

db_model.end()



DB켜짐
DB꺼짐


In [65]:
db_model = DbModel("main")
    
join_keys = ['pattern','storage'] #모델명으로 데이터 조인
b_merge = bunjang_index.drop(['id','bj_code','name'],axis=1).astype({'storage': 'str'}).\
    rename(columns={'key':'bunjang_key'})
j_merge = joongabi_index.drop(['id','brand_id','brand_name','model_id','model_name'],axis=1).\
    rename(columns={'model_code':'joongabi_key'})
m_merge = mintit_index.drop(['id','brand_id','brand_name','generation','model_name','retail_price','model_id']\
            ,axis=1).astype({'storage': 'str'}).\
    rename(columns={'model_code':'joongabi_key'})
p_merge = price_list.drop(['price_id','model_name','price'],axis=1).\
    rename(columns={'model_code':'price_list_key'})

df_OUTER_JOIN = pd.merge(b_merge, j_merge, left_on=join_keys, right_on=join_keys, how='outer')
df_OUTER_JOIN = pd.merge(df_OUTER_JOIN,m_merge, left_on=join_keys, right_on=join_keys, how='outer')
df_OUTER_JOIN = pd.merge(df_OUTER_JOIN,p_merge, left_on=join_keys, right_on=join_keys, how='outer')

result = df_OUTER_JOIN[['pattern','storage','mintit_key','bunjang_key','joongabi_key','price_list_key']].drop_duplicates()

result_for_join = result.rename(columns={'pattern':'pl_model_code'})
result_for_join = pd.merge(result_for_join,phone_list, left_on='pl_model_code', right_on='pl_model_code', how='inner')
# display(result_for_join[['pl_id','pl_model_name','pl_model_code','storage' ,'mintit_key', 'bunjang_key','joongabi_key','price_list_key']])

result_for_join = result_for_join[['pl_model_code','storage' ,'mintit_key', 'bunjang_key','joongabi_key','price_list_key']]

result_for_join.columns = ['original_code', 'storage', 'mintit_code','bunjang_code','joongabi_code','price_code']
result_for_join = result_for_join.replace("None",np.NAN)
# result_for_join.reset_index().drop('index',axis=1).drop_duplicates().to_sql(name='code_matching', con=db_model.engine, if_exists='append', index=False)
display(result_for_join)


db_model.end()

DB켜짐


,original_code,storage,mintit_code,bunjang_code,joongabi_code,price_code
0,SM-N986,256,SM-N986,DGTLSSSMN986N256GB,SM-N986,SM-N986N
1,SM-N981,256,SM-N981,DGTLSSSMN981N256GB,SM-N981,SM-N981N
2,SM-N976,512,SM-N976-512G,DGTLSSGN10P5G512GB,SM-N976_512G,SM-N976N
3,SM-N976,256,SM-N976-256G,DGTLSSGN10P5G256GB,SM-N976_256G,NaN
4,SM-N971,256,SM-N971-256G,DGTLSSGN105G256GB,SM-N971_256G,SM-N971N
5,SM-N960,512,SM-N960-512G,DGTLSSGN9512GB,SM-N960_512G,SM-N960N
6,SM-N960,128,SM-N960-128G,DGTLSSGN9128GB,SM-N960_128G,NaN
7,SM-N950,256,SM-N950-256G,DGTLSSGN8256GB,SM-N950_256G,NaN
8,SM-N950,64,SM-N950-64G,DGTLSSGN864GB,SM-N950_64G,SM-N950L
9,SM-G998,512,SM-G998-512G,DGTLSSSMG998N512GB,NaN,SM-G998N


DB꺼짐


In [71]:
# pd.merge(result_for_join,result_for_join, left_on=['original_code','storage'],right_on=['original_code','storage'], how='inner')
pd.merge(result_for_join, result_for_join, on=['original_code','storage'])

,original_code,storage,mintit_code_x,bunjang_code_x,joongabi_code_x,price_code_x,mintit_code_y,bunjang_code_y,joongabi_code_y,price_code_y
0,SM-N986,256,SM-N986,DGTLSSSMN986N256GB,SM-N986,SM-N986N,SM-N986,DGTLSSSMN986N256GB,SM-N986,SM-N986N
1,SM-N981,256,SM-N981,DGTLSSSMN981N256GB,SM-N981,SM-N981N,SM-N981,DGTLSSSMN981N256GB,SM-N981,SM-N981N
2,SM-N976,512,SM-N976-512G,DGTLSSGN10P5G512GB,SM-N976_512G,SM-N976N,SM-N976-512G,DGTLSSGN10P5G512GB,SM-N976_512G,SM-N976N
3,SM-N976,256,SM-N976-256G,DGTLSSGN10P5G256GB,SM-N976_256G,NaN,SM-N976-256G,DGTLSSGN10P5G256GB,SM-N976_256G,NaN
4,SM-N971,256,SM-N971-256G,DGTLSSGN105G256GB,SM-N971_256G,SM-N971N,SM-N971-256G,DGTLSSGN105G256GB,SM-N971_256G,SM-N971N
5,SM-N960,512,SM-N960-512G,DGTLSSGN9512GB,SM-N960_512G,SM-N960N,SM-N960-512G,DGTLSSGN9512GB,SM-N960_512G,SM-N960N
6,SM-N960,128,SM-N960-128G,DGTLSSGN9128GB,SM-N960_128G,NaN,SM-N960-128G,DGTLSSGN9128GB,SM-N960_128G,NaN
7,SM-N950,256,SM-N950-256G,DGTLSSGN8256GB,SM-N950_256G,NaN,SM-N950-256G,DGTLSSGN8256GB,SM-N950_256G,NaN
8,SM-N950,64,SM-N950-64G,DGTLSSGN864GB,SM-N950_64G,SM-N950L,SM-N950-64G,DGTLSSGN864GB,SM-N950_64G,SM-N950L
9,SM-G998,512,SM-G998-512G,DGTLSSSMG998N512GB,NaN,SM-G998N,SM-G998-512G,DGTLSSSMG998N512GB,NaN,SM-G998N
